## Notebook

Add julia functions by calling

In [ ]:
include("Julia_function/func_1.jl")

In a julia code env

In [ ]:
using Gridap

# Define computational domain
model = CartesianDiscreteModel((0,1,0,1), (4,4))
